In [ ]:
!pip install cdlib

  Using cached cdlib-0.2.4-py3-none-any.whl (208 kB)
  Using cached eva_lcd-0.1.1-py3-none-any.whl (9.2 kB)
  Using cached angel_cd-1.0.3-py3-none-any.whl (10 kB)
  Using cached bimlpa-0.1.2-py3-none-any.whl (7.0 kB)
  Using cached markov_clustering-0.0.6.dev0-py3-none-any.whl (6.3 kB)
  Using cached leidenalg-0.8.7-cp37-cp37m-manylinux2010_x86_64.whl (1.4 MB)
  Using cached dynetx-0.3.0-py3-none-any.whl (39 kB)
  Using cached wurlitzer-3.0.2-py3-none-any.whl (7.3 kB)
  Using cached nf1-0.0.4-py3-none-any.whl (18 kB)
  Using cached infomap-1.6.0.tar.gz (276 kB)
  Using cached python-Levenshtein-0.12.2.tar.gz (50 kB)
  Using cached PuLP-2.5.0-py3-none-any.whl (41.2 MB)
  Using cached thresholdclustering-1.1-py3-none-any.whl (5.3 kB)
  Using cached demon-2.0.6-py3-none-any.whl (7.3 kB)
  Using cached python_igraph-0.9.6-cp37-cp37m-manylinux2010_x86_64.whl (3.2 MB)
  Using cached karateclub-1.2.1.tar.gz (59 kB)
  Using cached chinese_whispers-0.7.5-py3-none-any.whl (7.6 kB)
  Using cached

In [ ]:
import json
import os
import networkx as nx
import pandas as pd 
from cdlib import NodeClustering, evaluation

In [ ]:
def get_communities(path, first_delimiter, delimiter, seperate_comm_id):
    comm_dict = dict()
    with open(path, "rt") as f:
        for row in f:
            node, row = row.rstrip().split(first_delimiter, 1)
            node = int(node)
            record = list(map(int, row.split(delimiter)))
            for comm_id in record:
                if comm_id in comm_dict.keys():
                    comm_dict[comm_id].append(node)
                else:
                    comm_dict[comm_id] = [node]
    communities = []
    for k, v in comm_dict.items():
        if not seperate_comm_id:
            v.append(k)
        communities.append(v)
    return communities

In [ ]:

def main():
    NMI_result = dict()
    Omega_result = dict()
    for N in [1000, 5000, 10000]:
        NMI_result[N] = dict()
        Omega_result[N] = dict()
        for mu in [0.1, 0.3]:
            NMI_result[N][mu] = {'NMI':[], 'NMI_max':[]}
            Omega_result[N][mu] = {'omega':[]}
            for om in range(2,9):
                communities = get_communities(os.path.join('N-{}-mu{:1.1f}-om{}-result.txt'.format(N, mu, om)), " ", " ", False)
                gt_communities = get_communities(os.path.join('N-{}-mu{:1.1f}-om{}-community.txt'.format(N, mu, om)), "\t", " ", True)
                df = pd.read_csv('N-{}-mu{:1.1f}-om{}.csv'.format(N, mu, om))
                G = nx.from_pandas_edgelist(df,source="From",target="To")
                coms = NodeClustering(communities, G, "", overlap=True)
                gt_coms = NodeClustering(gt_communities, G, "", overlap=True)
                nmi = evaluation.overlapping_normalized_mutual_information_LFK(coms, gt_coms)[0]
                nmi_max = evaluation.overlapping_normalized_mutual_information_MGH(coms, gt_coms)[0]
                omega = evaluation.omega(coms, gt_coms)[0]
                NMI_result[N][mu]['NMI'].append(nmi)
                NMI_result[N][mu]['NMI_max'].append(nmi_max)
                Omega_result[N][mu]['omega'].append(omega)

                #print("\nN: ", N,"mu: ", mu,"om: ", om,"NMI_result: ", NMI_result, "Omega_result: ", Omega_result)
        
        with open('LFR_nmi.json', 'w') as f:
            json.dump(NMI_result, f)
        with open('LFR_omega.json', 'w') as f:
            json.dump(Omega_result, f)
      


In [ ]:
"""
def main():
    NMI_result = dict()
    Omega_result = dict()
    N=10000
    mu=0.1
    om=2
    NMI_result[N] = dict()
    Omega_result[N] = dict()
        
    NMI_result[N][mu] = {'NMI':[], 'NMI_max':[]}
    Omega_result[N][mu] = {'omega':[]}
                
    communities = get_communities(os.path.join('N-{}-mu{:1.1f}-om{}-result.txt'.format(N, mu, om)), " ", " ", False)
    gt_communities = get_communities(os.path.join('N-{}-mu{:1.1f}-om{}-community.txt'.format(N, mu, om)), "\t", " ", True)
    df = pd.read_csv('N-{}-mu{:1.1f}-om{}.csv'.format(N, mu, om))
    G = nx.from_pandas_edgelist(df,source="From",target="To")
    coms = NodeClustering(communities, G, "", overlap=True)
    gt_coms = NodeClustering(gt_communities, G, "", overlap=True)
    #nmi = evaluation.overlapping_normalized_mutual_information_LFK(coms, gt_coms)[0]
    #nmi_max = evaluation.overlapping_normalized_mutual_information_MGH(coms, gt_coms)[0]
    omega = evaluation.omega(coms, gt_coms)[0]
    #NMI_result[N][mu]['NMI'].append(nmi)
    #NMI_result[N][mu]['NMI_max'].append(nmi_max)
    #Omega_result[N][mu]['omega'].append(omega)

    print(omega)
    
    #with open('LFR_nmi.json', 'w') as f:
    #json.dump(NMI_result, f)
    #with open('LFR_omega.json', 'w') as f:
    #json.dump(Omega_result, f)
    
"""

'\ndef main():\n    NMI_result = dict()\n    Omega_result = dict()\n    N=10000\n    mu=0.1\n    om=2\n    NMI_result[N] = dict()\n    Omega_result[N] = dict()\n        \n    NMI_result[N][mu] = {\'NMI\':[], \'NMI_max\':[]}\n    Omega_result[N][mu] = {\'omega\':[]}\n                \n    communities = get_communities(os.path.join(\'N-{}-mu{:1.1f}-om{}-result.txt\'.format(N, mu, om)), " ", " ", False)\n    gt_communities = get_communities(os.path.join(\'N-{}-mu{:1.1f}-om{}-community.txt\'.format(N, mu, om)), "\t", " ", True)\n    df = pd.read_csv(\'N-{}-mu{:1.1f}-om{}.csv\'.format(N, mu, om))\n    G = nx.from_pandas_edgelist(df,source="From",target="To")\n    coms = NodeClustering(communities, G, "", overlap=True)\n    gt_coms = NodeClustering(gt_communities, G, "", overlap=True)\n    #nmi = evaluation.overlapping_normalized_mutual_information_LFK(coms, gt_coms)[0]\n    #nmi_max = evaluation.overlapping_normalized_mutual_information_MGH(coms, gt_coms)[0]\n    omega = evaluation.omega(co

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
"""
from google.colab import files
uploaded = files.upload()
"""